## An IoT 'Sensor - Processor' Co-Simulation Example
> This example is aimed at simulating how sensing (and data conditioning) followed by processing data can be implemented as two concurrent processes. Between these two processes they share a circular buffer/memory among others for ensuring sensed data are stored and processed without any potential hazards.
> You will learn, how to

> 1/ create concurrent processes

> 2/ create thread safe locking variables including mutexes

Sensors in an IoT system continuously collect real-world data such as temperature, pressure, motion, or humidity. These sensors generate raw data in the form of electrical signals, which are then converted into digital values using an **Analog-to-Digital Converter (ADC)**. In this example, we will assume the details of ADC are masked and we are able to generate random sensing data for simulation purposes.

The sensor data is stored in a **shared memory** (in this case a circular buffer with a capacity of 10 data samples max). In our case, we will assume two sensors are writing data to this shared memory using synchronization mechanisms to prevent data corruption.  

An **IoT processor** (which could be a microcontroller, edge device, or cloud server agent) retrieves this stored data, processes it (for example for filtering, anomaly detection, or machine learning models, and then interprets) for decision-making. This processed information is either **displayed to users**, **triggering an action**, or **transmitted to cloud services** for further analysis. In our example, we will just assume each data are arithmetically processed for data conditioning.

This seamless flow from data generation to interpretation enables **real-time monitoring and automation** in IoT applications such as smart homes, healthcare, and industrial automation. 🚀

The following is a block diagram of the Sensor - Processor Co-Simulation system:

<center><img src="Sensor-Processor.png" width="40%" /></center>

### Petri Net representation of the Co-Simulation Environment

(Please note for demonstration purposes - we have made the following PN descriptions a lot more detailed - but you can choose to work at a higher level of abstraction given the rubrics for your coursework. Please note in your coursework - no specific requirements were mentioned about safe or unsafe PN descriptions.)

Below we generated two (high-level) Petri Net representations of this example: the first model assumes an **unsafe model** and the second model assumes a **1-safe model** maximally. For each model, we will discuss their design choices with respect to the model complexity and safety tradeoffs.

> **_Note:_** Our Petri Net models will be based on the Python code implementation shown below.
>
> Also, to help reduce the size of our 1-safe Petri Net, we will only model up to three memories and make use of Workcraft's features, e.g. *proxy places*.

#### Unsafe Petri Net Design

<center><img src="unsafe-petri-net.png" width="50%" /></center>

In our unsafe Petri Net model, we have:

- **Seven transitions**. Three transitions called *sensor1*, *sensor2* and *processor1* that each represent 'Sensor 1', 'Sensor 2' and 'Processor' respectively. Another three transitions, where two transitions called *put_1* and *put_2* represent when the sensors add data to the buffer/memory and one transition called *get* represent when the processor retrieves data from the buffer/memory. Lastly, we also have an additional transition called *reset_mux* that replenishes the token back at place *mux* to reset the system for the next call.

- **Ten places**. Three places that represent the semaphores used in the following implementation called *mux* (representing our mutex), *empty* (representing when there is data to be written), and *full* (representing when there is data to be retrieved. Another three places called *sens1_do_op*, *sens2_do_op* and *proc1_do_op* that represent when *sensor1*, *sensor2* and *processor1* are in operation after receiving the token from *mux* respectively. Three more places called *sens1_cnt*, *sens2_cnt* and *proc1_cnt* used to count the number of items handled by *sensor1*, *sensor2* and *processor1* respectively. Finally, one place called *op_done* used to reset the system via *reset_mux*.

> **_Note:_** As *mux*, *empty* and *full* implement the three semaphores in our implementation, it is important to take note of the numbers assigned to each of them as these will act as our **tokens**. In this case, *mux* will have one token but *empty* will have 10 tokens and *full* will have 0 tokens.

To capture the behaviour of the implementation below, we:

- First connect *mux* to *sensor1*, *sensor2* and *processor1*, as well as from *sensor1*, *sensor2* and *processor1* to their respectively *do_op* and *cnt* places.
  
- We also connect the *put* transitions and *get* transition to form the basic producer-consumer Petri Net (with one exception being that we have two put transitions and 10 tokens instead of 1).

- To allow our system to reset, a connection is made from the *put* and *get* transitions to the place *op_done* before we connect an arc from it to *reset_mux* and from *reset_mux* back to *mux*.

- Lastly, we also connect read arcs between the *put* transitions and *empty* place to ensure the sensors can only add data when there is empty space, as well as between the *get* transition and *full* transition to ensure the processor can only retrieve data when there is occupied space.

> **_Note:_** The final unsafe Petri Net design is very compact and easily captures the behaviour of our Python implementation below. However, despite this, it is worth mentioning that the unsafe Petri Net gives no indicator of what order the data is being handled in -- If you carefully examine the Python code, you will realise that data is actually handled sequentially in a round-robin order.
>
> But, we can, of course, simply assume that the data will be handled in the order that it has been placed by the sensors. **However, do mind that you must explicitly state this when you do such an approach**.  
>
> As *mux*, *empty* and *full* implement the three semaphores in our implementation, it is important to take note of the numbers assigned to each of them as these will act as our **tokens**. In this case, *mux* will have one token but *empty* will have 10 tokens and *full* will have 0 tokens.

#### 1-safe Petri Net Design

<center><img src="safe-petri-net.png" width="95%" /></center>

In our 1-safe Petri Net model, we have:

- **Nineteen transitions**. Three dedicated transitions of each 'Sensor 1', 'Sensor 2' and 'Processor' called *sensor1*, *sensor2* and *processor1* followed by an assigned number from 1 to 3 (totalling Nine transitions). Three *put_..._1*, *put_..._2* and *get_...* transitions for each of the three memories (also totalling Nine transitions). Lastly, like the Unsafe Petri Net, an additional transition called *reset_mux* that replenishes the token back at place *mux* to reset the system for the next call.

- **Seventeen places**. Three dedicated places of each *empty* and *full* semaphore for each of the three memories (totalling Six places) followed by one place representing the *mux*. Three places for each of the *put* and *get* sides to form a **queue** between the memroies (totalling Six places). Another three places called *sens1_do_op*, *sens2_do_op* and *proc1_do_op*, as well another place called *op_done* that have the same behaviour as their counterpart in the Unsafe Petri Net design.

> **_Note:_** As the number of memories are explicitated, the number of *empty* and *full* places depend on the number of memories there are in the design, e.g. if you have 20 memories then you must have 20 *empty* places and 20 *full* places, with a token assigned to each *empty* place initially.

To capture the behaviour of the implementation below, we:

- Each *sensor1*, *sensor2* and *processor1* transition must be connected by a producing arc from place *mux* and by a consuming arc to their respective *do_op* place.

- Additionally, transitions *sensor1*, *sensor2* and *processor1* must each be connected with a read arc between it and their corresponding memory's empty/full place (e.g. *sensor1/2_1* connects to *empty01*, *processor1_1* to *full01*, *sensor1/2_2* to *empty02*, etc.), and a read arc between it and their corresponding memory's queueing place (e.g. *sensor1/2_1* connects to *p1*, *processor1_1* to *g1*, *sensor1/2_2* to *p2*, etc.). This is to ensure the sensors can only add data when there is empty space and the processors can only retrieve data when there is occupied space.

- Each of the *do_op* places connect to their respective *put/get* transitions.

- Each memory, their pair of *put* transitions and *get* transition is connected with their empty and full places to form the basic producer-consumer Petri net segment.

- Every *put* and *get* transition is connected to the *op_done* place, followed by a connection from *op_done* to *reset_mux* and from *reset_mux* to *mux* to allow the system to reset.

> **_Note:_** Unlike our unsafe Petri Net design, our 1-safe Petri Net design is much larger even when only considering three memories (meaning if we do all ten memories or even more, our design can exponentially grow!).
>
> However, the advantages over the unsafe Petri net is that this model can be easily verified and implemented on to Asynchronous platforms, and we can see a near 1-to-1 behaviour between it and the Python implementation and the order of how data is handled.
>
> Unfortunately (unless you do some very clever modelling), there is not an easier way to make a very clean Safe Petri net -- This is especially the case if you want to **explictate the memories being occupied/taken in accordance with the sensors/processor**.

In [ ]:
import threading
import time

# Shared Memory variables
CAPACITY = 10
buffer = [-1 for i in range(CAPACITY)]
    # range(CAPACITY): Generates a sequence of numbers from 0 to CAPACITY - 1.
    # for i in range(CAPACITY): Iterates over each number in that sequence (though i is unused).
    # -1 for i in range(CAPACITY): Assigns -1 to each element in the list.
in_index = 0
out_index = 0

# Declaring Mutexes and thread-safe signals
mutex = threading.Semaphore()
empty = threading.Semaphore(CAPACITY)
full = threading.Semaphore(0)

# Producer Thread Class
class Sensor(threading.Thread):
  def run(self):

    global CAPACITY, buffer, in_index, out_index
    global mutex, empty, full

    items_produced = 0
    counter = 0

    while items_produced < 20:
      empty.acquire()
      mutex.acquire()

      counter += 1
      buffer[in_index] = counter
      print("Sensor", str(f'{self.name=}'), "sensed data : ", str(counter), "in memory location: ", str(in_index+1))
      in_index = (in_index + 1)%CAPACITY

      mutex.release()
      full.release()

      time.sleep(1)

      items_produced += 1

# Consumer Thread Class
class Processor(threading.Thread):
  def run(self):

    global CAPACITY, buffer, in_index, out_index, counter
    global mutex, empty, full

    items_consumed = 0

    while items_consumed < 20:
      full.acquire()
      mutex.acquire()

      item = buffer[out_index]
      print("Processor", str(f'{self.name=}'), "processed data : ", str(item), "from memory location: ", str(out_index+1))
      out_index = (out_index + 1)%CAPACITY
      #print("Consumer consumed item : ", item)

      mutex.release()
      empty.release()

      time.sleep(2.5)

      items_consumed += 1

# Creating Threads
sensor1 = Sensor()
sensor2 = Sensor()
processor1 = Processor()

# Starting Threads
sensor1.start()
sensor2.start()
processor1.start()

# Waiting for threads to complete
sensor1.join()
sensor2.join()
processor1.join()

Sensor self.name='Thread-78' sensed data :  1 in memory location:  1
Processor self.name='Thread-80' processed data :  1 from memory location:  1
Sensor self.name='Thread-79' sensed data :  1 in memory location:  2
Sensor self.name='Thread-78' sensed data :  2 in memory location:  3
Sensor self.name='Thread-79' sensed data :  2 in memory location:  4
Sensor self.name='Thread-79' sensed data :  3 in memory location:  5
Sensor self.name='Thread-78' sensed data :  3 in memory location:  6
Processor self.name='Thread-80' processed data :  1 from memory location:  2
Sensor self.name='Thread-79' sensed data :  4 in memory location:  7
Sensor self.name='Thread-78' sensed data :  4 in memory location:  8
Sensor self.name='Thread-78' sensed data :  5 in memory location:  9
Sensor self.name='Thread-79' sensed data :  5 in memory location:  10
Processor self.name='Thread-80' processed data :  2 from memory location:  3
Sensor self.name='Thread-79' sensed data :  6 in memory location:  1
Sensor se

Note that **Sensor 1** and **Sensor 2** run concurrently in separate threads while **Processor** runs as an independent thread. As expected, they share a common buffer/memory which is implemented using **a mutually exclusive access to all reads and writes**.